# RAG의 기초 

하는 이유 - 최신정보같이 없는 정보에 대한 할루시네이션이 발생할수있음

진행순서 : 문서로드 - 텍스트 스플릿(청크단위) - 나눠진 텍스트 임베딩(수치화) - 벡터 저장 -질문에 해당 되는 문서 검색

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [7]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("../data/data.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 5


### 내용확인

In [ ]:
print(docs[2].page_content)

### metadata 확인

In [ ]:
docs[2].__dict__

In [13]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 25


In [14]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [15]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [ ]:
for doc in vectorstore.similarity_search("하향식설계"):
    print(doc.page_content)

In [17]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [ ]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("요구사항 개발 프로세스의 순서는 무엇인가요?")

In [19]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [20]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [21]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "요구사항의 유형의 비기능적요구는 무엇이 있나요?"
response = chain.invoke(question)
print(response)

비기능적 요구에는 성능, 보안, 품질, 안정성 등이 있습니다.
